In [1]:
import numpy as np
import pandas as pd
import pickle
from keras.datasets import cifar10

2023-07-18 16:27:07.580455: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-18 16:27:08.321894: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load training and testing data separately
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


In [3]:
# Reshape x_train from 4D to 2D array (number of samples, width*height*channels)
x_train = x_train.reshape(x_train.shape[0], -1)

# Reshape y_train to 1D array
y_train = y_train.reshape(-1)

# Combine training data and labels into a single numpy array for easier manipulation
train_data = np.column_stack((x_train, y_train))

# Randomly shuffle the training data
np.random.shuffle(train_data)


In [4]:
data_size = len(x_train)
data_size

50000

In [5]:
def count_classes(y):
    unique, counts = np.unique(y, return_counts=True)
    return dict(zip(unique, counts))

# Testing the function
class_counts = count_classes(y_train)
print(class_counts)


{0: 5000, 1: 5000, 2: 5000, 3: 5000, 4: 5000, 5: 5000, 6: 5000, 7: 5000, 8: 5000, 9: 5000}


In [6]:

# # Number of chunks
# n_chunks = 5

# # Generate sizes following a power-law distribution
# sizes = np.random.zipf(1.5, n_chunks)

# # Normalize the sizes so that their sum equals the number of training samples
# sizes = (sizes / sizes.sum() * len(train_data)).astype(int)

# # Ensure that the sum of sizes is equal to the total number of training samples
# sizes[-1] += len(train_data) - sizes.sum()



# sizes

In [7]:
chunk_sizes = [0.1, 0.15, 0.2, 0.25, 0.3]

sizes = np.array([int(chunk_size*data_size) for chunk_size in chunk_sizes])


sizes

array([ 5000,  7500, 10000, 12500, 15000])

In [8]:
def split_data_unbalanced(data, labels, sizes):
    total_samples = len(labels)
    classes, counts = np.unique(labels, return_counts=True)
    num_classes = len(classes)

    # # Convert sizes from proportions to absolute values
    # sizes = [int(size * total_samples) for size in sizes]

    # Indices of each class in the data
    indices = [np.where(labels == c)[0].tolist() for c in classes]

    # Number of chunks
    num_chunks = len(sizes)
    
    # List to store chunks of data and labels
    chunks_data, chunks_labels = [], []

    # Iterate over each chunk
    for i, size in enumerate(sizes):
        chunk_data, chunk_labels = [], []
        
        # Ensure each class is represented in each chunk
        for c in range(num_classes):
            if len(indices[c]) > 0:
                idx = indices[c].pop(0)
                chunk_data.append(data[idx])
                chunk_labels.append(labels[idx])

        # Fill the rest of the chunk with randomly selected samples from all classes
        all_indices = [idx for class_indices in indices for idx in class_indices]
        np.random.shuffle(all_indices)
        for _ in range(size - num_classes):
            if len(all_indices) > 0:
                idx = all_indices.pop(0)
                chunk_data.append(data[idx])
                chunk_labels.append(labels[idx])

        # Add the current chunk to the list of chunks
        chunks_data.append(np.array(chunk_data))
        chunks_labels.append(np.array(chunk_labels))

    return chunks_data, chunks_labels

# Testing the function
chunks_data, chunks_labels = split_data_unbalanced(x_train, y_train, sizes)

# Checking the distribution of labels in each chunk
for i, chunk_labels in enumerate(chunks_labels):
    print(f"Chunk {i+1}:")
    unique, counts = np.unique(chunk_labels, return_counts=True)
    print(dict(zip(unique, counts)))


Chunk 1:
{0: 507, 1: 485, 2: 484, 3: 553, 4: 480, 5: 479, 6: 512, 7: 511, 8: 478, 9: 511}
Chunk 2:
{0: 761, 1: 760, 2: 781, 3: 733, 4: 738, 5: 754, 6: 757, 7: 729, 8: 760, 9: 727}
Chunk 3:
{0: 994, 1: 1013, 2: 1024, 3: 980, 4: 966, 5: 1034, 6: 1005, 7: 1048, 8: 984, 9: 952}
Chunk 4:
{0: 1258, 1: 1302, 2: 1242, 3: 1236, 4: 1202, 5: 1270, 6: 1213, 7: 1210, 8: 1321, 9: 1246}
Chunk 5:
{0: 1480, 1: 1478, 2: 1532, 3: 1517, 4: 1505, 5: 1494, 6: 1548, 7: 1487, 8: 1487, 9: 1472}


In [9]:
def split_data_unbalanced(data, labels, sizes):
    total_samples = len(labels)
    classes, counts = np.unique(labels, return_counts=True)
    num_classes = len(classes)

    # # Convert sizes from proportions to absolute values
    # sizes = [int(size * total_samples) for size in sizes]

    # Indices of each class in the data
    indices = [np.where(labels == c)[0].tolist() for c in classes]

    # List to store chunks of data and labels
    chunks_data, chunks_labels = [], []

    # Iterate over each chunk
    for i, size in enumerate(sizes):
        chunk_data, chunk_labels = [], []

        # Iterate over each class
        for c in range(num_classes):
            # Make sure we have enough samples for this class
            num_samples = min(len(indices[c]), max(int(size / num_classes), 1))

            # If we don't have enough samples, raise an error
            if num_samples > len(indices[c]):
                raise ValueError(f"Not enough samples in class {c} for chunk {i}")
            
            sample_indices = np.random.choice(indices[c], num_samples, replace=False)
            indices[c] = list(set(indices[c]) - set(sample_indices))
            
            chunk_data.extend(data[sample_indices])
            chunk_labels.extend(labels[sample_indices])

        chunks_data.append(np.array(chunk_data))
        chunks_labels.append(np.array(chunk_labels))

    return chunks_data, chunks_labels

# Testing the function
chunks_data, chunks_labels = split_data_unbalanced(x_train, y_train, sizes)

# Checking the distribution of labels in each chunk
for i, chunk_labels in enumerate(chunks_labels):
    print(f"Chunk {i+1}:")
    unique, counts = np.unique(chunk_labels, return_counts=True)
    print(dict(zip(unique, counts)))


Chunk 1:
{0: 500, 1: 500, 2: 500, 3: 500, 4: 500, 5: 500, 6: 500, 7: 500, 8: 500, 9: 500}
Chunk 2:
{0: 750, 1: 750, 2: 750, 3: 750, 4: 750, 5: 750, 6: 750, 7: 750, 8: 750, 9: 750}
Chunk 3:
{0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}
Chunk 4:
{0: 1250, 1: 1250, 2: 1250, 3: 1250, 4: 1250, 5: 1250, 6: 1250, 7: 1250, 8: 1250, 9: 1250}
Chunk 5:
{0: 1500, 1: 1500, 2: 1500, 3: 1500, 4: 1500, 5: 1500, 6: 1500, 7: 1500, 8: 1500, 9: 1500}


Chunk 1:
{0: 5000, 1: 5000, 2: 5000, 3: 5000, 4: 5000, 5: 5000, 6: 5000, 7: 5000, 8: 5000, 9: 5000}
Chunk 2:
{}
Chunk 3:
{}
Chunk 4:
{}
Chunk 5:
{}


In [12]:
data = x_train
labels = y_train

In [14]:
total_samples = len(labels)
classes, counts = np.unique(labels, return_counts=True)
num_classes = len(classes)

total_samples, classes, counts, num_classes

(50000,
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]),
 10)

In [15]:
sizes

array([ 5000,  7500, 10000, 12500, 15000])

In [10]:
import numpy as np

def split_data_unbalanced(data, labels, sizes):
    # total_samples = len(labels)
    classes, counts = np.unique(labels, return_counts=True)
    num_classes = len(classes)

    # Convert sizes from proportions to absolute values
    # sizes = [int(size * total_samples) for size in sizes]

    # Indices of each class in the data
    indices = [np.where(labels == c)[0].tolist() for c in classes]

    # List to store chunks of data and labels
    chunks_data, chunks_labels = [], []

    # Minimum number of samples of each class in each chunk
    min_samples = [int(0.05 * size) for size in sizes]

    # Initial distribution of samples to each chunk
    for c in range(num_classes):
        np.random.shuffle(indices[c])
        start = 0
        for i, size in enumerate(sizes):
            end = start + min_samples[i]
            chunks_data.append(data[indices[c][start:end]])
            chunks_labels.append(labels[indices[c][start:end]])
            start = end
        indices[c] = indices[c][start:]  # Remaining samples of this class

    # Distribute the remaining samples
    remaining_sizes = [size - len(chunk) for size, chunk in zip(sizes, chunks_data)]
    for i, size in enumerate(remaining_sizes):
        while size > 0:
            for c in range(num_classes):
                if size <= 0:
                    break
                if len(indices[c]) > 0:
                    chunks_data[i] = np.append(chunks_data[i], data[indices[c][0]])
                    chunks_labels[i] = np.append(chunks_labels[i], labels[indices[c][0]])
                    indices[c] = indices[c][1:]
                    size -= 1

    return chunks_data, chunks_labels

# Testing the function
chunks_data, chunks_labels = split_data_unbalanced(x_train, y_train, sizes)

# Checking the distribution of labels in each chunk
for i, chunk_labels in enumerate(chunks_labels):
    print(f"Chunk {i+1}:")
    unique, counts = np.unique(chunk_labels, return_counts=True)
    print(dict(zip(unique, counts)))


KeyboardInterrupt: 

In [20]:
# Indices of each class in the data
indices = [np.where(labels == c)[0].tolist() for c in classes]

# List to store chunks of data and labels
chunks_data, chunks_labels = [], []

# Minimum number of samples of each class in each chunk
min_samples = [int(0.05 * size) for size in sizes]


In [23]:
import numpy as np

def split_data_unbalanced(data, labels, sizes):
    classes, counts = np.unique(labels, return_counts=True)
    num_classes = len(classes)
    total_samples = len(labels)

    # Calculate minimum samples per class per chunk based on the 5% rule
    min_samples_per_chunk = [int(0.05 * size) for size in sizes]

    # Indices of each class in the data
    indices = [np.where(labels == c)[0] for c in classes]

    # List to store chunks of data and labels
    chunks_data, chunks_labels = [], []

    # Distribute minimum number of samples of each class to each chunk
    for i, size in enumerate(sizes):
        chunk_data, chunk_labels = [], []
        for c in range(num_classes):
            if len(indices[c]) < min_samples_per_chunk[i]:
                raise ValueError(f"Not enough samples in class {c} for chunk {i}")
            sample_indices = np.random.choice(indices[c], min_samples_per_chunk[i], replace=False)
            indices[c] = list(set(indices[c]) - set(sample_indices))
            chunk_data.extend(data[sample_indices])
            chunk_labels.extend(labels[sample_indices])
        chunks_data.append(np.array(chunk_data))
        chunks_labels.append(np.array(chunk_labels))

    # Distribute remaining samples to chunks
    remaining_samples = total_samples - sum([len(chunk) for chunk in chunks_data])
    for i, size in enumerate(sizes):
        remaining_size = size - len(chunks_data[i])
        while remaining_size > 0 and remaining_samples > 0:
            for c in range(num_classes):
                if len(indices[c]) > 0:
                    sample_index = indices[c].pop(0)
                    chunks_data[i] = np.append(chunks_data[i], data[sample_index])
                    chunks_labels[i] = np.append(chunks_labels[i], labels[sample_index])
                    remaining_size -= 1
                    remaining_samples -= 1
                if remaining_size <= 0:
                    break

    return chunks_data, chunks_labels

# Testing the function
chunks_data, chunks_labels = split_data_unbalanced(x_train, y_train, sizes)

# Checking the distribution of labels in each chunk
for i, chunk_labels in enumerate(chunks_labels):
    print(f"Chunk {i+1}:")
    unique, counts = np.unique(chunk_labels, return_counts=True)
    print(dict(zip(unique, counts)))


KeyboardInterrupt: 

In [11]:
import numpy as np

def split_data_unbalanced(data, labels, sizes):
    classes, counts = np.unique(labels, return_counts=True)
    num_classes = len(classes)
    total_samples = len(labels)

    # Calculate minimum samples per class per chunk based on the 5% rule
    min_samples_per_chunk = [int(0.05 * size) for size in sizes]

    # Indices of each class in the data
    indices = [np.where(labels == c)[0] for c in classes]

    # List to store chunks of data and labels
    chunks_data, chunks_labels = [], []

    # Distribute minimum number of samples of each class to each chunk
    for i, size in enumerate(sizes):
        chunk_data, chunk_labels = [], []
        for c in range(num_classes):
            if len(indices[c]) < min_samples_per_chunk[i]:
                raise ValueError(f"Not enough samples in class {c} for chunk {i}")
            sample_indices = np.random.choice(indices[c], min_samples_per_chunk[i], replace=False)
            indices[c] = list(set(indices[c]) - set(sample_indices))
            chunk_data.extend(data[sample_indices])
            chunk_labels.extend(labels[sample_indices])
        chunks_data.append(np.array(chunk_data))
        chunks_labels.append(np.array(chunk_labels))

    # Distribute remaining samples to chunks
    remaining_samples = total_samples - sum([len(chunk) for chunk in chunks_data])
    for i, size in enumerate(sizes):
        remaining_size = size - len(chunks_data[i])
        while remaining_size > 0 and remaining_samples > 0:
            for c in range(num_classes):
                if len(indices[c]) > 0:
                    sample_index = indices[c].pop(0)
                    chunks_data[i] = np.append(chunks_data[i], data[sample_index])
                    chunks_labels[i] = np.append(chunks_labels[i], labels[sample_index])
                    remaining_size -= 1
                    remaining_samples -= 1
                if remaining_size <= 0:
                    break

    return chunks_data, chunks_labels

# Testing the function
chunks_data, chunks_labels = split_data_unbalanced(x_train, y_train, sizes)

# Checking the distribution of labels in each chunk
for i, chunk_labels in enumerate(chunks_labels):
    print(f"Chunk {i+1}:")
    unique, counts = np.unique(chunk_labels, return_counts=True)
    print(dict(zip(unique, counts)))


Chunk 1:
{0: 500, 1: 500, 2: 500, 3: 500, 4: 500, 5: 500, 6: 500, 7: 500, 8: 500, 9: 500}
Chunk 2:
{0: 750, 1: 750, 2: 750, 3: 750, 4: 750, 5: 750, 6: 750, 7: 750, 8: 750, 9: 750}
Chunk 3:
{0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}
Chunk 4:
{0: 1250, 1: 1250, 2: 1250, 3: 1250, 4: 1250, 5: 1250, 6: 1250, 7: 1250, 8: 1250, 9: 1250}
Chunk 5:
{0: 1500, 1: 1500, 2: 1500, 3: 1500, 4: 1500, 5: 1500, 6: 1500, 7: 1500, 8: 1500, 9: 1500}


In [9]:
# Create an array of indices at which to split the training data
split_indices = np.cumsum(sizes)[:-1]

# Split the training data into chunks of different sizes
chunks = np.split(train_data, split_indices)


In [8]:

# Function to get label distribution in a chunk
def get_label_distribution(chunk):
    # The label is in the last column
    labels = chunk[:, -1]
    unique_labels, counts = np.unique(labels, return_counts=True)
    return dict(zip(unique_labels, counts))


# Function to get label proportions in a chunk
def get_label_proportions(label_distribution, chunk_size):
    proportions = {}
    for label, count in label_distribution.items():
        proportions[label] = count / chunk_size
    return proportions




In [9]:
import os
folder = "5_chunks_2"
if not os.path.exists(folder):
    os.makedirs(folder)


In [14]:
noniid_folder = "noniid"
noniid_folder = os.path.join(folder, noniid_folder)
if not os.path.exists(noniid_folder):
    os.makedirs(noniid_folder)


In [15]:

# Store size and label distribution of each chunk, and save each chunk as a pickle file
unbalanced_chunk_info = []
# Now, make the distribution unbalanced for each chunk before saving
for i, unbalanced_chunk in enumerate(chunks):

    info = {}
    info['chunk'] = i+1
    label_distribution = get_label_distribution(unbalanced_chunk)
    chunk_size =len(unbalanced_chunk)
    info['size'] = chunk_size
    info['label_distribution'] = label_distribution

    proportions = get_label_proportions(label_distribution, chunk_size)
    info['label_proportions'] = proportions

    unbalanced_chunk_info.append(info)

    # Save unbalanced chunk as a pickle file
    with open(f'{noniid_folder}/chunk_{i+1}.pickle', 'wb') as f:
        pickle.dump(unbalanced_chunk, f)


ValueError: Cannot take a larger sample than population when 'replace=False'

In [16]:
# Convert list of dictionaries to DataFrame for better visualization
noniid_df = pd.DataFrame(unbalanced_chunk_info)

# print dataframe
print(noniid_df)

Empty DataFrame
Columns: []
Index: []


In [15]:
# Save dataframe to csv
noniid_df.to_csv(f"{noniid_folder}/chunks_info.csv", index=False)


In [68]:
df.size

30

In [69]:
df

,chunk,size,label_distribution
0,1,2173,"{0: 211, 1: 226, 2: 215, 3: 207, 4: 232, 5: 21..."
1,2,2173,"{0: 209, 1: 212, 2: 228, 3: 230, 4: 223, 5: 24..."
2,3,8695,"{0: 867, 1: 915, 2: 852, 3: 840, 4: 858, 5: 87..."
3,4,2173,"{0: 203, 1: 201, 2: 211, 3: 242, 4: 230, 5: 20..."
4,5,4347,"{0: 447, 1: 425, 2: 397, 3: 417, 4: 462, 5: 45..."
5,6,2173,"{0: 216, 1: 210, 2: 226, 3: 224, 4: 209, 5: 19..."
6,7,13043,"{0: 1231, 1: 1256, 2: 1333, 3: 1319, 4: 1328, ..."
7,8,2173,"{0: 228, 1: 240, 2: 221, 3: 188, 4: 210, 5: 19..."
8,9,10869,"{0: 1166, 1: 1086, 2: 1085, 3: 1120, 4: 1056, ..."
9,10,2181,"{0: 222, 1: 229, 2: 232, 3: 213, 4: 192, 5: 24..."


In [71]:
df.label_distribution


0    {0: 211, 1: 226, 2: 215, 3: 207, 4: 232, 5: 21...
1    {0: 209, 1: 212, 2: 228, 3: 230, 4: 223, 5: 24...
2    {0: 867, 1: 915, 2: 852, 3: 840, 4: 858, 5: 87...
3    {0: 203, 1: 201, 2: 211, 3: 242, 4: 230, 5: 20...
4    {0: 447, 1: 425, 2: 397, 3: 417, 4: 462, 5: 45...
5    {0: 216, 1: 210, 2: 226, 3: 224, 4: 209, 5: 19...
6    {0: 1231, 1: 1256, 2: 1333, 3: 1319, 4: 1328, ...
7    {0: 228, 1: 240, 2: 221, 3: 188, 4: 210, 5: 19...
8    {0: 1166, 1: 1086, 2: 1085, 3: 1120, 4: 1056, ...
9    {0: 222, 1: 229, 2: 232, 3: 213, 4: 192, 5: 24...
Name: label_distribution, dtype: object

In [21]:
import numpy as np
import os
import pickle
import pandas as pd

from tensorflow.keras import datasets
from collections import defaultdict, Counter
from sklearn.model_selection import train_test_split

# Define the proportion of each chunk
chunk_sizes = [0.1, 0.15, 0.2, 0.25, 0.3]

# Define the proportion of each category within each chunk
category_dist_in_chunks = [
    [0.3, 0.2, 0.15, 0.1, 0.05, 0.05, 0.05, 0.05, 0.025, 0.025],
    [0.1, 0.2, 0.1, 0.15, 0.1, 0.1, 0.1, 0.05, 0.05, 0.1],
    [0.1, 0.05, 0.15, 0.1, 0.2, 0.1, 0.1, 0.05, 0.1, 0.05],
    [0.05, 0.1, 0.05, 0.2, 0.1, 0.15, 0.1, 0.1, 0.05, 0.1],
    [0.1, 0.05, 0.1, 0.05, 0.15, 0.1, 0.2, 0.1, 0.1, 0.05]
]

# Normalize the proportions within each chunk
category_dist_in_chunks = [np.array(dist)/sum(dist) for dist in category_dist_in_chunks]

def save_to_pkl(images, labels, path):
    """Save the images and labels to a Pickle file."""
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'wb') as f:
        # Combine images and labels into one list
        data = [np.append(image.flatten(), label) for image, label in zip(images, labels)]
        pickle.dump(data, f)
    print(f'Dataset saved to {path}')

# Load CIFAR10 data
(train_images, train_labels), _ = datasets.cifar10.load_data()

# Create a dictionary where the keys are the labels and the values are the images
data_dict = defaultdict(list)
for image, label in zip(train_images, train_labels):
    data_dict[label[0]].append(image)

chunk_info = []
# Create chunks
for i in range(5):
    chunk_images = []
    chunk_labels = []
    total_samples = sum([len(images) for images in data_dict.values()])
    chunk_size = int(total_samples * chunk_sizes[i])
    
    for label, images in data_dict.items():
        n_samples = int(len(images) * category_dist_in_chunks[i][label])
        # Split data for this category
        chunk, data_dict[label] = train_test_split(images, train_size=n_samples, shuffle=True)
        chunk_images.extend(chunk)
        chunk_labels.extend([label]*len(chunk))
    
    # Correct for rounding errors
    diff = chunk_size - len(chunk_images)
    if diff != 0:
        label, images = max(data_dict.items(), key=lambda x: len(x[1]))
        extra_samples, data_dict[label] = train_test_split(images, train_size=diff, shuffle=True)
        chunk_images.extend(extra_samples)
        chunk_labels.extend([label]*len(extra_samples))
    
    # Save the chunk
    save_to_pkl(chunk_images, chunk_labels, f'data2/chunk_{i+1}.pkl')

    # Collect the chunk information
    chunk_info.append({
        'chunk': i+1,
        'size': len(chunk_images),
        'label_distribution': dict(Counter(chunk_labels))
    })

# Save the chunk information into a CSV file
df = pd.DataFrame(chunk_info)
df.to_csv('data2/chunk_info.csv', index=False)


Dataset saved to data2/chunk_1.pkl
Dataset saved to data2/chunk_2.pkl
Dataset saved to data2/chunk_3.pkl


ValueError: train_size=4463 should be either positive and smaller than the number of samples 3765 or a float in the (0, 1) range

In [3]:
import os, sys
root = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(root)

root

'/home/vtn_ubuntu/ttu/spring23/working_project/AsynFl'

In [4]:
from asynfed.client.local_model_upload_info import LocalModelUpdateInfo

In [5]:
first_info = LocalModelUpdateInfo()

In [7]:
first_info.update(remote_weight_path= "cloud/123.pkl", local_weight_path='./123.pkl', filename= '123.pkl',
                  global_version_used= 10, train_acc= 0.20, train_loss= 0.6)

first_info.to_dict()

{'local_weight_path': './123.pkl',
 'remote_weight_path': 'cloud/123.pkl',
 'filename': '123.pkl',
 'global_version_used': 10,
 'new_update': True,
 'train_acc': 0.2,
 'train_loss': 0.6}

In [8]:
first_info_clone = LocalModelUpdateInfo(**first_info.to_dict())

first_info_clone.to_dict()

{'local_weight_path': './123.pkl',
 'remote_weight_path': 'cloud/123.pkl',
 'filename': '123.pkl',
 'global_version_used': 10,
 'new_update': True,
 'train_acc': 0.2,
 'train_loss': 0.6}

In [10]:
first_info.update(remote_weight_path= "cloud/124.pkl", local_weight_path='./124.pkl', filename= '124.pkl',
                  global_version_used= 10, train_acc= 0.10, train_loss= 0.5)

first_info_clone.to_dict(), first_info.to_dict() 

({'local_weight_path': './123.pkl',
  'remote_weight_path': 'cloud/123.pkl',
  'filename': '123.pkl',
  'global_version_used': 10,
  'new_update': True,
  'train_acc': 0.2,
  'train_loss': 0.6},
 {'local_weight_path': './124.pkl',
  'remote_weight_path': 'cloud/124.pkl',
  'filename': '124.pkl',
  'global_version_used': 10,
  'new_update': True,
  'train_acc': 0.1,
  'train_loss': 0.5})